## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):

    - pymorphy2

    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [ ]:
!pip install nltk
!pip install pymorphy2
!pip install gensim

In [17]:
import pandas as pd
import nltk
from nltk import word_tokenize
import pymorphy2
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import os
import json
import re
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch
import torch.nn.functional as F
from torch import FloatTensor, LongTensor
import math
from sklearn.linear_model import LogisticRegression

tqdm.pandas()
nltk.download('punkt')
morph = pymorphy2.MorphAnalyzer()
df = pd.read_csv('news_train.txt', sep='\t',header=None )
df.columns = ['category', 'title', 'text']


/home/lissrbay/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
[nltk_data] Downloading package punkt to /home/lissrbay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### №1

In [18]:
df['text'] = df['text'].progress_apply(word_tokenize)
df['text'] = df['text'].progress_apply(lambda x: [word for word in x if word.isalpha()])
df['text'] = df['text'].progress_apply(lambda x: [morph.parse(word)[0].normal_form for word in x])
df.head()


100%|██████████| 15000/15000 [09:34<00:00, 26.09it/s]


,category,title,text
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,"[нападать, вашингтон, кэпиталзти, александр, о..."
1,culture,Рекордно дорогую статую майя признали подделкой,"[власть, мексика, объявить, подделка, статуя, ..."
2,science,Samsung представила флагман в защищенном корпусе,"[южнокорейский, samsung, анонсировать, защитит..."
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,"[комитет, кдк, рфс, снять, дисквалификация, с,..."
4,media,Hopes & Fears объединится с The Village,"[hopes, fears, объявить, о, свой, слияние, с, ..."


### №2

In [21]:
w2v_model = Word2Vec()
sentences = df['text'].values
w2v_model.build_vocab(sentences, progress_per=10000)


In [22]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)


(64809694, 79175640)

Пример семантической связи: близкие слова для слова вратарь.

In [8]:
w2v_model.wv.most_similar(positive=["вратарь"])

[('голкипер', 0.8509472012519836),
 ('форвард', 0.8013597726821899),
 ('полузащитник', 0.7753452062606812),
 ('хоккеист', 0.7597266435623169),
 ('нападать', 0.7316120862960815),
 ('денисов', 0.7236849069595337),
 ('малкин', 0.7207446694374084),
 ('кокорин', 0.7188119292259216),
 ('семин', 0.7075212001800537),
 ('пингвинс', 0.7069272994995117)]

Пример семантической связи: близкие слова для слова вклад и те, которые по смыслу далеки от темы банка.

In [27]:
w2v_model.wv.most_similar(positive=['вклад'])

[('депозит', 0.6161470413208008),
 ('кредит', 0.5344643592834473),
 ('страхование', 0.5078973770141602),
 ('взнос', 0.49820154905319214),
 ('ссуда', 0.4761122465133667),
 ('ставка', 0.4754984974861145),
 ('банковский', 0.47037452459335327),
 ('страхов', 0.4685913622379303),
 ('кредитование', 0.46608319878578186),
 ('актив', 0.46394848823547363)]

In [28]:
w2v_model.wv.most_similar(positive=['вклад'], negative=["банк"])

[('выдающийся', 0.4104170501232147),
 ('дебют', 0.4048970937728882),
 ('мастерство', 0.3994008004665375),
 ('уайлёд', 0.3963795006275177),
 ('режиссура', 0.37563467025756836),
 ('отчисление', 0.3611883521080017),
 ('подвиг', 0.35683977603912354),
 ('задекларировать', 0.35437875986099243),
 ('валери', 0.35391169786453247),
 ('веха', 0.352017879486084)]

### №3

In [29]:
X_train = df['text'].values
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
vectorizer.fit_transform([x for x in X_train])
idf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [34]:
def embedding(sentence, embedding_size=100):
    vec = np.zeros(embedding_size).reshape((1, embedding_size))
    count = 0.
    for word in sentence:
        try:
            vec += w2v_model[word].reshape((1, embedding_size)) * idf[word]
            count += 1.
        except KeyError: 
            continue
        vec /= (count if count > 0 else 1)
    return vec

df['wv'] = df['text'].progress_apply(lambda x: embedding(x))
df.head()

  0%|          | 0/15000 [00:00<?, ?it/s]/home/lissrbay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
100%|██████████| 15000/15000 [00:24<00:00, 624.41it/s]


,category,title,text,wv
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,"[нападать, вашингтон, кэпиталзти, александр, о...","[[0.7729980481114793, -3.510473666990057, -0.6..."
1,culture,Рекордно дорогую статую майя признали подделкой,"[власть, мексика, объявить, подделка, статуя, ...","[[-0.5589148749907812, -0.127827702910595, 1.7..."
2,science,Samsung представила флагман в защищенном корпусе,"[южнокорейский, samsung, анонсировать, защитит...","[[-0.0800700051819577, -0.4852414146230063, 1...."
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,"[комитет, кдк, рфс, снять, дисквалификация, с,...","[[-1.7253159720506241, -3.6982621303412007, 2...."
4,media,Hopes & Fears объединится с The Village,"[hopes, fears, объявить, о, свой, слияние, с, ...","[[1.7165771966827088, 0.11026712150677391, -0...."


In [35]:
def prepare_data(df):
    X, y = df['wv'].values, df['category']
    X_ = []
    for i in X:
        X_.append(np.array(i).reshape(100))
    X = np.array(X_)
    y = LabelEncoder().fit_transform(y)
    return X, y
X, y = prepare_data(df)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
classifier_svm = SVC(class_weight='balanced', gamma='auto')
params = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': [1, 1e-1, 1e-2]
}
gs = GridSearchCV(classifier_svm, params, cv=5, n_jobs=7, verbose=2)
gs.fit(X_train, y_train)
print('Best parameters:', gs.best_params_)
y_pred = gs.predict(X_test)
print('Accuracy', accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:  1.7min
[Parallel(n_jobs=7)]: Done  45 out of  45 | elapsed:  4.0min finished


Best parameters: {'C': 1, 'kernel': 'rbf'}
Accuracy 0.851


In [37]:
classifier_lr = LogisticRegression(class_weight='balanced', penalty='elasticnet', solver='saga')
params = {
    'l1_ratio': [1, 1e-1, 1e-2, 1e-3],
    'C': [1, 1e-1, 1e-2]
}
gs = GridSearchCV(classifier_lr, params, cv=5, n_jobs=7, verbose=2)
gs.fit(X_train, y_train)
print('Best parameters:', gs.best_params_)

y_pred = gs.predict(X_test)
print('Accuracy', accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:  2.5min
[Parallel(n_jobs=7)]: Done  60 out of  60 | elapsed:  5.3min finished
/home/lissrbay/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/lissrbay/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Best parameters: {'C': 0.1, 'l1_ratio': 0.001}
Accuracy 0.8316666666666667
